In [1]:
%matplotlib qt5
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
from scipy.interpolate import interp1d
from utils.plotting import calculate_error_bars
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import os
import peakutils
import matplotlib
from matplotlib.lines import Line2D
from utils.plotting_visuals import makes_plots_pretty
import pickle 
from utils.post_processing_utils import remove_exps_after_manipulations, remove_bad_recordings
from utils.regression.linear_regression_utils import get_first_x_sessions
import pandas as pd
from utils.reaction_time_utils import get_bpod_trial_nums_per_session
from utils.plotting import calculate_error_bars

In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def get_all_mice_data(experiments_to_process):
    exp_numbers = []
    mice = []
    for index, experiment in experiments_to_process.iterrows():
        mouse = experiment['mouse_id']
        date = experiment['date']
        dates = experiments_to_process[experiments_to_process['mouse_id'] == mouse]['date'].values
        session_starts = get_bpod_trial_nums_per_session(mouse, dates)
        session_ind = np.where(dates == date)[0][0]
        session_start_trial = session_starts[session_ind]
        saving_folder = 'W:\\photometry_2AC\\processed_data\\for_figure\\' + mouse + '\\'
        save_filename = mouse + '_' + date + '_' + 'aligned_traces_correct_incorrect.p'

        sorted_exps = pd.to_datetime(
            experiments_to_process[experiments_to_process['mouse_id'] == mouse]['date']).sort_values(ignore_index=True)
        date_as_dt = pd.to_datetime(date)
        exp_number = sorted_exps[sorted_exps == date_as_dt].index[0]
        exp_numbers.append(exp_number)
        with open(saving_folder + save_filename, "rb") as f:
            content = pickle.load(f)
        print(mouse, date)
        if index == 0:
            correct = content.choice_data.contra_correct_data.sorted_traces[:,int(160000/2-20000):int(160000/2+20000)]
            incorrect = content.choice_data.contra_incorrect_data.sorted_traces[:,int(160000/2-20000):int(160000/2+20000)]
            time_stamps = content.choice_data.contra_correct_data.time_points[int(160000/2-20000):int(160000/2+20000)]
            correct_trial_nums = content.choice_data.contra_correct_data.trial_nums + session_start_trial
            incorrect_trial_nums = content.choice_data.contra_incorrect_data.trial_nums + session_start_trial
            reaction_times = content.choice_data.contra_incorrect_data.reaction_times
            correct_reaction_times = content.choice_data.contra_correct_data.reaction_times
            correct_peaks = np.asarray(content.choice_data.contra_correct_data.trial_peaks)
            incorrect_peaks = np.asarray(content.choice_data.contra_incorrect_data.trial_peaks)
        else:
            correct = np.vstack([correct, content.choice_data.contra_correct_data.sorted_traces[:,int(160000/2-20000):int(160000/2+20000)]])
            incorrect = np.vstack([incorrect,content.choice_data.contra_incorrect_data.sorted_traces[:,int(160000/2-20000):int(160000/2+20000)]])
            correct_trial_nums = np.concatenate((correct_trial_nums, content.choice_data.contra_correct_data.trial_nums + session_start_trial))
            incorrect_trial_nums = np.concatenate((incorrect_trial_nums, content.choice_data.contra_incorrect_data.trial_nums + session_start_trial))
            reaction_times = np.concatenate((reaction_times, content.choice_data.contra_incorrect_data.reaction_times))
            correct_reaction_times = np.concatenate((correct_reaction_times, content.choice_data.contra_correct_data.reaction_times))
            correct_peaks = np.concatenate((correct_peaks, np.asarray(content.choice_data.contra_correct_data.trial_peaks)))
            incorrect_peaks = np.concatenate((incorrect_peaks, np.asarray(content.choice_data.contra_incorrect_data.trial_peaks)))
    return correct, incorrect, correct_peaks, incorrect_peaks, correct_trial_nums, incorrect_trial_nums, correct_reaction_times, reaction_times, time_stamps



In [3]:

mouse_ids = ['SNL_photo22']
site = 'tail'

experiment_record = pd.read_csv('W:\\photometry_2AC\\experimental_record.csv')
experiment_record['date'] = experiment_record['date'].astype(str)
clean_experiments = remove_exps_after_manipulations(experiment_record, mouse_ids)
all_experiments_to_process = clean_experiments[
    (clean_experiments['mouse_id'].isin(mouse_ids)) & (clean_experiments['recording_site'] == site)].reset_index(
    drop=True)
experiments_to_process = remove_bad_recordings(all_experiments_to_process).reset_index(drop=True)


removing SNL_photo22: ['20200829' '20200830' '20200831' '20200908' '20200910' '20200911'
 '20200915' '20200917' '20200918' '20200921' '20201007' '20201008'
 '20201009']
Int64Index([], dtype='int64')


In [4]:
correct, incorrect, correct_peaks, incorrect_peaks, correct_trial_nums, incorrect_trial_nums, correct_reaction_times, incorrect_reaction_times, time_stamps = get_all_mice_data(experiments_to_process)

SNL_photo22 20200806
SNL_photo22 20200808
SNL_photo22 20200810
SNL_photo22 20200812
SNL_photo22 20200814
SNL_photo22 20200816
SNL_photo22 20200818
SNL_photo22 20200820
SNL_photo22 20200822
SNL_photo22 20200824
SNL_photo22 20200826
SNL_photo22 20200827
SNL_photo22 20200828


In [ ]:
all_reaction_times = np.concatenate([incorrect_reaction_times, correct_reaction_times])
max_reaction_time = int(np.max(all_reaction_times))+1


In [594]:
fig, axs = plt.subplots(1,1)
axs.hist(incorrect_reaction_times, bins=np.linspace(0, max_reaction_time, 100) , density=True, color='red', alpha=0.2)
axs.hist(correct_reaction_times, bins=np.linspace(0, max_reaction_time, 100), density=True, color='green', alpha=0.2)
axs.set_ylabel(' % of trials')
axs.set_xlabel('reaction time (s)')
axs.axvline(np.median(all_reaction_times), color='k', lw=1)
axs.axvline(np.quantile(all_reaction_times, 0.75), color='k', linestyle='--', lw=1)
axs.axvline(np.quantile(all_reaction_times, 0.25), color='k', linestyle='--', lw=1)
axs.axvline(np.median(correct_reaction_times), color='g', lw=1)
axs.axvline(np.quantile(correct_reaction_times, 0.75), color='g', linestyle='--', lw=1)
axs.axvline(np.quantile(correct_reaction_times, 0.25), color='g', linestyle='--', lw=1)
axs.axvline(np.median(incorrect_reaction_times), color='r', lw=1)
axs.axvline(np.quantile(incorrect_reaction_times, 0.75), color='r', linestyle='--', lw=1)
axs.axvline(np.quantile(incorrect_reaction_times, 0.25), color='r', linestyle='--', lw=1)
axs.set_xlim([0,6])

(0.0, 6.0)

In [17]:
upper_quartile = np.quantile(correct_reaction_times, 0.75)
lower_quartile = np.quantile(correct_reaction_times, 0.25)

In [18]:
incorrect_reaction_times_valid_inds = np.where(incorrect_reaction_times <=upper_quartile)
correct_reaction_times_valid_inds = np.where(correct_reaction_times <=upper_quartile)

In [648]:
correct_reaction_times = correct_reaction_times[correct_reaction_times_valid_inds]
incorrect_reaction_times = incorrect_reaction_times[incorrect_reaction_times_valid_inds]

fig, axs = plt.subplots(1,1)
axs.hist(incorrect_reaction_times, bins=np.linspace(0, max_reaction_time, 100) , density=True, color='red', alpha=0.2)
axs.hist(correct_reaction_times, bins=np.linspace(0, max_reaction_time, 100), density=True, color='green', alpha=0.2)
axs.set_ylabel(' % of trials')
axs.set_xlabel('reaction time (s)')
axs.axvline(np.median(correct_reaction_times), color='g', lw=1)
axs.axvline(np.quantile(correct_reaction_times, 0.75), color='g', linestyle='--', lw=1)
axs.axvline(np.quantile(correct_reaction_times, 0.25), color='g', linestyle='--', lw=1)
axs.axvline(np.median(incorrect_reaction_times), color='r', lw=1)
axs.axvline(np.quantile(incorrect_reaction_times, 0.75), color='r', linestyle='--', lw=1)
axs.axvline(np.quantile(incorrect_reaction_times, 0.25), color='r', linestyle='--', lw=1)
axs.set_xlim([0,6])

(0.0, 6.0)

In [205]:
plt.figure()
plt.scatter(incorrect_trial_nums, incorrect_reaction_times, color='r', s=2)
plt.scatter(correct_trial_nums, correct_reaction_times, color='g', s=2)

In [207]:
plt.scatter(incorrect_reaction_times, incorrect_peaks, color='r', s=5)
plt.scatter(correct_reaction_times, correct_peaks, color='g', s=2, alpha=0.2)

In [12]:
np.mean(incorrect_peaks[mid_incorrect_inds])

1.055835958212447

In [15]:
np.mean(correct_peaks[mid_correct_inds])

1.1683922039861983

In [19]:
incorrect_trial_nums = incorrect_trial_nums[incorrect_reaction_times_valid_inds]
incorrect = incorrect[incorrect_reaction_times_valid_inds]

In [20]:
correct_trial_nums = correct_trial_nums[correct_reaction_times_valid_inds]
correct = correct[correct_reaction_times_valid_inds]

In [21]:
incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, incorrect_trial_nums))[0]
correct_inds = find_nearest_trials(incorrect_trial_nums, correct_trial_nums)

In [132]:
plt.plot(np.mean(incorrect[incorrect_inds], axis=0))
plt.plot(np.mean(correct[correct_inds], axis=0))

In [22]:
max_trials = np.max(np.concatenate((incorrect_trial_nums, correct_trial_nums)))
early_incorrect_trials = incorrect_trial_nums[ incorrect_trial_nums < int(max_trials/3)]
mid_incorrect_trials = incorrect_trial_nums[np.logical_and(incorrect_trial_nums < int(max_trials/3)*2, incorrect_trial_nums > int(max_trials/3))]
late_incorrect_trials = incorrect_trial_nums[np.logical_and(incorrect_trial_nums <= max_trials, incorrect_trial_nums > int(max_trials/3)*2)]

In [23]:
early_incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, early_incorrect_trials))[0]
mid_incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, mid_incorrect_trials))[0]
late_incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, late_incorrect_trials))[0]

In [7]:
def find_nearest_trials(target_trials, other_trials):
    differences = (target_trials.reshape(1,-1) - other_trials.reshape(-1,1))
    indices = np.abs(differences).argmin(axis=0)
    residual = np.diagonal(differences[indices,])
    return indices

In [24]:
early_correct_inds = find_nearest_trials(early_incorrect_trials, correct_trial_nums)
mid_correct_inds = find_nearest_trials(mid_incorrect_trials, correct_trial_nums)
late_correct_inds = find_nearest_trials(late_incorrect_trials, correct_trial_nums)

In [148]:
def get_traces_and_mean(all_traces, inds):
    traces = all_traces[inds, :]
    mean_trace = np.mean(traces, axis=0)
    return traces, mean_trace

In [149]:
def plot_trace(all_traces, inds, time_stamps, ax, color='green'):
    traces, mean_trace = get_traces_and_mean(all_traces, inds)

    ax.plot(time_stamps, mean_trace, color=color)
    error_bar_lower, error_bar_upper = calculate_error_bars(mean_trace,
                                                            traces,
                                                            error_bar_method='sem')
    ax.fill_between(time_stamps, error_bar_lower, error_bar_upper, alpha=0.5,
                     facecolor=color, linewidth=0)
    ax.set_xlabel('time(s)')
    ax.set_ylabel('z-scored fluorescence')


In [150]:
def get_peak(all_traces, inds, median_reaction_time):
    traces, mean_trace = get_traces_and_mean(all_traces, inds)
    half_way = int(traces.shape[1]/2)
    trace_from_event = mean_trace[half_way:half_way + int(1000*(median_reaction_time))]
    print(half_way + int(1000*(median_reaction_time)))
    trial_peak_inds = peakutils.indexes(trace_from_event.flatten('F'))
    if trial_peak_inds.shape[0] > 0 or len(trial_peak_inds > 1):
        trial_peak_inds = trial_peak_inds[0] 
        trial_peaks = trace_from_event.flatten('F')[trial_peak_inds]
    else:
        trial_peak_inds = np.argmax(trace_from_event) 
        trial_peaks = np.max(trace_from_event)
    trial_peak_inds += half_way
    return trial_peaks, trial_peak_inds

In [151]:
median_reaction_time = np.median(all_reaction_times)
sd_reaction_times = np.std(all_reaction_times)

In [152]:
limit =  median_reaction_time + sd_reaction_times *2 + median_reaction_time

In [153]:
fig, axs = plt.subplots(1,3, sharey=True)
axs[0].set_title('Early')
plot_trace(correct, early_correct_inds, time_stamps, axs[0], color='green')
trial_peaks, trial_peak_inds = get_peak(correct, early_correct_inds, limit)
axs[0].scatter(time_stamps[trial_peak_inds], trial_peaks, color='green')
plot_trace(incorrect, early_incorrect_inds, time_stamps, axs[0], color='red')
trial_peaks, trial_peak_inds = get_peak(incorrect, early_incorrect_inds, limit)
axs[0].scatter(time_stamps[trial_peak_inds], trial_peaks, color='red')

axs[1].set_title('Middle')
plot_trace(correct, mid_correct_inds, time_stamps, axs[1], color='green')
trial_peaks, trial_peak_inds = get_peak(correct, mid_correct_inds, limit)
axs[1].scatter(time_stamps[trial_peak_inds], trial_peaks, color='green')
plot_trace(incorrect, mid_incorrect_inds, time_stamps, axs[1], color='red')
trial_peaks, trial_peak_inds = get_peak(incorrect, mid_incorrect_inds, limit)
axs[1].scatter(time_stamps[trial_peak_inds], trial_peaks, color='red')

axs[2].set_title('Late')
plot_trace(correct, late_correct_inds, time_stamps, axs[2], color='green')
trial_peaks, trial_peak_inds = get_peak(correct, late_correct_inds, limit)
axs[2].scatter(time_stamps[trial_peak_inds], trial_peaks, color='green')
plot_trace(incorrect, late_incorrect_inds, time_stamps, axs[2], color='red')
trial_peaks, trial_peak_inds = get_peak(incorrect, late_incorrect_inds, limit)
axs[2].scatter(time_stamps[trial_peak_inds], trial_peaks, color='red')
plt.tight_layout()
print(early_incorrect_inds.shape[0], mid_incorrect_inds.shape[0], late_incorrect_inds.shape[0])

22151
22151
22151
22151
22151
22151
143 36 10


Not trial matching


In [ ]:
max_trials = np.max(np.concatenate((incorrect_trial_nums, correct_trial_nums)))
early_incorrect_trials = incorrect_trial_nums[ incorrect_trial_nums < int(max_trials/3)]
mid_incorrect_trials = incorrect_trial_nums[np.logical_and(incorrect_trial_nums < int(max_trials/3)*2, incorrect_trial_nums > int(max_trials/3))]
late_incorrect_trials = incorrect_trial_nums[np.logical_and(incorrect_trial_nums <= max_trials, incorrect_trial_nums > int(max_trials/3)*2)]
early_incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, early_incorrect_trials))[0]
mid_incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, mid_incorrect_trials))[0]
late_incorrect_inds = np.nonzero(np.in1d(incorrect_trial_nums, late_incorrect_trials))[0]

In [ ]:
early_correct_trials = correct_trial_nums[ correct_trial_nums < int(max_trials/3)]
mid_correct_trials = correct_trial_nums[np.logical_and(correct_trial_nums < int(max_trials/3)*2, correct_trial_nums > int(max_trials/3))]
late_correct_trials = correct_trial_nums[np.logical_and(correct_trial_nums <= max_trials, correct_trial_nums > int(max_trials/3)*2)]
early_correct_inds = np.nonzero(np.in1d(correct_trial_nums, early_correct_trials))[0]
mid_correct_inds = np.nonzero(np.in1d(correct_trial_nums, mid_correct_trials))[0]
late_correct_inds = np.nonzero(np.in1d(correct_trial_nums, late_correct_trials))[0]

In [450]:
fig, axs = plt.subplots(1,3, sharey=True)
axs[0].set_title('Early')
plot_trace(correct, early_correct_inds, time_stamps, axs[0], color='green')
trial_peaks, trial_peak_inds = get_peak(correct, early_correct_inds, limit)
axs[0].scatter(time_stamps[trial_peak_inds], trial_peaks, color='green')
plot_trace(incorrect, early_incorrect_inds, time_stamps, axs[0], color='red')
trial_peaks, trial_peak_inds = get_peak(incorrect, early_incorrect_inds, limit)
axs[0].scatter(time_stamps[trial_peak_inds], trial_peaks, color='red')

axs[1].set_title('Middle')
plot_trace(correct, mid_correct_inds, time_stamps, axs[1], color='green')
trial_peaks, trial_peak_inds = get_peak(correct, mid_correct_inds, limit)
axs[1].scatter(time_stamps[trial_peak_inds], trial_peaks, color='green')
plot_trace(incorrect, mid_incorrect_inds, time_stamps, axs[1], color='red')
trial_peaks, trial_peak_inds = get_peak(incorrect, mid_incorrect_inds, limit)
axs[1].scatter(time_stamps[trial_peak_inds], trial_peaks, color='red')

axs[2].set_title('Late')
plot_trace(correct, late_correct_inds, time_stamps, axs[2], color='green')
trial_peaks, trial_peak_inds = get_peak(correct, late_correct_inds, limit)
axs[2].scatter(time_stamps[trial_peak_inds], trial_peaks, color='green')
plot_trace(incorrect, late_incorrect_inds, time_stamps, axs[2], color='red')
trial_peaks, trial_peak_inds = get_peak(incorrect, late_incorrect_inds, limit)
axs[2].scatter(time_stamps[trial_peak_inds], trial_peaks, color='red')
plt.tight_layout()
print(early_incorrect_inds.shape[0], mid_incorrect_inds.shape[0], late_incorrect_inds.shape[0])

23535
23535
23535
23535
23535
23535
146 52 14


In [321]:
time_stamps[trial_peak_inds]

-1.8765617285108034

In [297]:
incorrect_trial_nums.shape

(212,)

In [191]:
late_incorrect_trials

array([ 7009.,  7127.,  7197.,  7391.,  7483.,  8160.,  8284.,  8345.,
        8350.,  9288.,  9657.,  9829., 10064., 10084.])

In [192]:
max_trials

10107.0

In [ ]:
dir = 'W:\\photometry_2AC\\processed_data\\for_figure\\'
file_name = 'correct_incorrect_group_data_' + site +'.npz'
np.savez(dir + file_name, correct=ipsi_choice, incorrect=incorrect, time_stamps=time_stamps)



In [302]:
correct = content.choice_data.contra_correct_data.mean_trace
incorrect = content.choice_data.contra_incorrect_data.mean_trace

In [303]:
plt.plot(correct[int(160000/2-20000):int(160000/2+20000)], color='green')
plt.plot(incorrect[int(160000/2-20000):int(160000/2+20000)], color='red')

In [304]:
# TO DO: find the nearest correct trial to incorrect trial and match them for average

In [305]:
incorrect_trials = content.choice_data.contra_incorrect_data.trial_nums
correct_trials =  content.choice_data.contra_correct_data.trial_nums

In [306]:
incorrect_trials.shape

(94,)

In [307]:
differences = (incorrect_trials.reshape(1,-1) - correct_trials.reshape(-1,1))

In [308]:
indices = np.abs(differences).argmin(axis=0)
residual = np.diagonal(differences[indices,])
indices.shape

(94,)

In [309]:
a = plt.plot(np.mean(content.choice_data.contra_incorrect_data.sorted_traces[:,int(160000/2-20000):int(160000/2+20000)].T, axis=1), color='red')
b = plt.plot(np.mean(content.choice_data.contra_correct_data.sorted_traces[indices,int(160000/2-20000):int(160000/2+20000)].T, axis=1), color='green')

In [390]:
a = np.full([2,3], 'a')


In [391]:
a

array([['a', 'a', 'a'],
       ['a', 'a', 'a']], dtype='<U1')